In [1]:
import pandas as pd
import numpy as np
import time
import collections
from sklearn.model_selection import (
    LeaveOneOut,
    GridSearchCV
)
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

# Using filtered vsd.csv (genes = 50) for testing purposes

In [2]:
df = pd.read_csv("~/Documents/GMU/Research_Jafri/tcell_model/work/vsd_corrected_ml_test.csv", index_col = 0)
colData = pd.read_csv("~/Documents/GMU/Research_Jafri/tcell_model/work/samples.tsv", sep = "\t", index_col = 0)

In [3]:
X = df.T
y = colData["state"] == "Th1"

### 0. Functions for feature selection, model training, and model evaluation

In [5]:
def feature_sel(X_train, y_train, estimator):

    # Create sfs
    sfs = SequentialFeatureSelector(
        estimator = estimator,
        n_features_to_select = 10
    )

    # Fit sfs
    sfs.fit(X_train, y_train)
    
    # Save list of features 
    estimator_features = sfs.get_support()
    features = (list(X_train.columns[estimator_features]))

    return features


def model_train(X_train, y_train):

    # Create classifier models and tune parameters
    
    # LogisticRegression
    clf_lr = GridSearchCV(
        estimator = LogisticRegression(C = 1.0, solver = "liblinear", penalty = "l1", max_iter = 300),
        param_grid = {
            "C": [i for i in range(1, 100, 1)] # test C from 1-100 incrementing by 1
        },
        refit = True,
        scoring = "accuracy",
        cv = LOOCV,
        n_jobs = -1
    ).fit(X_train, y_train)

    # RandomForest
    clf_rf = GridSearchCV(
        estimator = RandomForestClassifier(n_estimators = 100),
        param_grid = {
            "n_estimators": [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
        },
        refit = True,
        scoring = "accuracy",
        cv = LOOCV,
        n_jobs = -1
    ).fit(X_train, y_train)

    # SVM
    clf_svm = GridSearchCV(
        estimator = SVC(C = 1.0, kernel = "linear"),
        param_grid = {
            "C": [i for i in range(1, 100, 1)] # test C from 1-100 incrementing by 1
        },
        refit = True,
        scoring = "accuracy",
        cv = LOOCV,
        n_jobs = -1
    ).fit(X_train, y_train)

    return (clf_lr, clf_rf, clf_svm)

def model_eval(models, X_test, y_test):

    scores = []

    # Generate X_test predictions using different models
    y_pred_lr = models[0].predict(X_test)
    y_pred_rf = models[1].predict(X_test)
    y_pred_svm = models[2].predict(X_test)

    # Store accuracy scores
    scores.append(accuracy_score(y_test, y_pred_lr))
    scores.append(accuracy_score(y_test, y_pred_rf))
    scores.append(accuracy_score(y_test, y_pred_svm))

    return scores


### 1. Feature selection (LASSO and RandomForest)

In [39]:
# Set start time
start_time = time.perf_counter()

# Initialize leave-one-out cross validator
LOOCV = LeaveOneOut()

# Initialize lists for features per fold
features_lasso = []
features_rf = []

# Split using LOOCV
for fold_idx, (train_idx, test_idx) in enumerate(LOOCV.split(X)):

    # Filter X and y given training and testing indices
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    print("Starting fold:", fold_idx)

    # LASSO feature selection (per fold)
    print("Starting LASSO feature selection...")
    
    # Set estimator
    estimator_lasso = LogisticRegression(solver = "liblinear", penalty = "l1", max_iter = 300, random_state = 42)

    # Perform feature selection with LASSO estimator
    feature_selection_lasso = feature_sel(X_train, y_train, estimator_lasso)

    # Save features (per fold)
    features_lasso.append(feature_selection_lasso)
    
    # RandomForest feature selection (per fold)
    print("Starting RandomForest feature selection...")

    # Set estimator
    estimator_rf = RandomForestClassifier(n_estimators = 100, random_state = 42)

    # Perform feature selection with rf estimator
    feature_selection_rf = feature_sel(X_train, y_train, estimator_rf)

    # Save features (per fold)
    features_rf.append(feature_selection_rf) 

    print("Feature selection complete for fold:", fold_idx)
    print()

# Find the frequency of selected features and select the top 10

# Create counters
counter_lasso = collections.Counter()
counter_rf = collections.Counter()

# LASSO
for features in features_lasso:
    counter_lasso.update(features)

features_lasso_10 = [feature for feature, count in counter_lasso.most_common(10)]
features_lasso_10

# RandomForest
for features in features_rf:
    counter_rf.update(features)

features_rf_10 = [feature for feature, count in counter_rf.most_common(10)]
features_rf_10

# Print elapsed time
end_time = time.perf_counter()
print("Elapsed time:", end_time - start_time)

Starting fold: 0
Starting LASSO feature selection...
Starting RandomForest feature selection...
Feature selection complete for fold: 0

Starting fold: 1
Starting LASSO feature selection...
Starting RandomForest feature selection...
Feature selection complete for fold: 1

Starting fold: 2
Starting LASSO feature selection...
Starting RandomForest feature selection...
Feature selection complete for fold: 2

Starting fold: 3
Starting LASSO feature selection...
Starting RandomForest feature selection...
Feature selection complete for fold: 3

Starting fold: 4
Starting LASSO feature selection...
Starting RandomForest feature selection...
Feature selection complete for fold: 4

Starting fold: 5
Starting LASSO feature selection...
Starting RandomForest feature selection...
Feature selection complete for fold: 5

Starting fold: 6
Starting LASSO feature selection...
Starting RandomForest feature selection...
Feature selection complete for fold: 6

Starting fold: 7
Starting LASSO feature selectio

### 2. Model Training, Tuning, and Evaluation

In [68]:
# Set start time
start_time = time.perf_counter()

# Initialize leave-one-out cross validator
LOOCV = LeaveOneOut()

# Filter training and testing data to include selected features
X_lasso = X[features_lasso_10]
X_rf = X[features_rf_10]

# Initialize lists for storing accuracy scores per fold
acc_lr = []
acc_rf = []
acc_svm = []

acc_lasso_lr = []
acc_lasso_rf = []
acc_lasso_svm = []

acc_rf_lr = []
acc_rf_rf = []
acc_rf_svm = []


# Split using LOOCV
for fold_idx, (train_idx, test_idx) in enumerate(LOOCV.split(X)):

    # Filter X and y given training and testing indices 
    # for control (no feature selction), LASSO, and RandomForest
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    X_train_lasso, X_test_lasso = X_lasso.iloc[train_idx], X_lasso.iloc[test_idx]
    X_train_rf, X_test_rf = X_rf.iloc[train_idx], X_rf.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    print("Starting fold:", fold_idx)

    # Train models with no feature selection
    print("Starting model training with no feature selection...")
    models_crtl = model_train(X_train, y_train)
    
    # Train models with LASSO filtered data
    print("Starting model training with LASSO filtered data...")
    models_lasso = model_train(X_train_lasso, y_train)

    # Train models with RandomForest filtered data
    print("Starting model training with RandomForest filtered data...")
    models_rf = model_train(X_train_rf, y_train)
    
    # Evaluate control models
    acc_scores = model_eval(models_crtl, X_test, y_test)
    acc_lr.append(acc_scores[0])
    acc_rf.append(acc_scores[1])
    acc_svm.append(acc_scores[2])

    # Evaluate LASSO filtered models
    acc_scores_lasso = model_eval(models_lasso, X_test_lasso, y_test)
    acc_lasso_lr.append(acc_scores_lasso[0])
    acc_lasso_rf.append(acc_scores_lasso[1])
    acc_lasso_svm.append(acc_scores_lasso[2])

    # Evaluate RandomForest filtered models
    acc_scores_rf = model_eval(models_rf, X_test_rf, y_test)
    acc_rf_lr.append(acc_scores_rf[0])
    acc_rf_rf.append(acc_scores_rf[1])
    acc_rf_svm.append(acc_scores_rf[2])

    print()

# Print accuracy scores
print("Accuracy score for LogisticRegression using no feature selection:", round(np.mean(acc_lr), 3))
print("Accuracy score for RandomForest using no feature selection:", round(np.mean(acc_rf), 3))
print("Accuracy score for SVM using no feature selection:", round(np.mean(acc_svm), 3))
print()
print("Accuracy score for LogisticRegression using LASSO feature selection:", round(np.mean(acc_lasso_lr), 3))
print("Accuracy score for RandomForest using LASSO feature selection:", round(np.mean(acc_lasso_rf), 3))
print("Accuracy score for SVM using LASSO feature selection:", round(np.mean(acc_lasso_svm), 3))
print()
print("Accuracy score for LogisticRegression using RandomForest feature selection:", round(np.mean(acc_rf_lr), 3))
print("Accuracy score for RandomForest using RandomForest feature selection:", round(np.mean(acc_rf_rf), 3))
print("Accuracy score for SVM using RandomForest feature selection:", round(np.mean(acc_rf_svm), 3))
print()    

# Print elapsed time
end_time = time.perf_counter()
print("Elapsed time:", end_time - start_time)

Starting fold: 0
Starting model training with no feature selection
Starting model training with LASSO filtered data
Starting model training with RandomForest filtered data

Starting fold: 1
Starting model training with no feature selection
Starting model training with LASSO filtered data
Starting model training with RandomForest filtered data

Starting fold: 2
Starting model training with no feature selection
Starting model training with LASSO filtered data
Starting model training with RandomForest filtered data

Starting fold: 3
Starting model training with no feature selection
Starting model training with LASSO filtered data
Starting model training with RandomForest filtered data

Starting fold: 4
Starting model training with no feature selection
Starting model training with LASSO filtered data
Starting model training with RandomForest filtered data

Starting fold: 5
Starting model training with no feature selection
Starting model training with LASSO filtered data
Starting model trai

# Full-scale vsd.csv

## Import counts from R and metadata

In [2]:
df = pd.read_csv("~/Documents/GMU/Research_Jafri/tcell_model/work/vsd_corrected_ml.csv", index_col = 0)
colData = pd.read_csv("~/Documents/GMU/Research_Jafri/tcell_model/work/samples.tsv", sep = "\t", index_col = 0)
top_genes_list = pd.read_csv("~/Documents/GMU/Research_Jafri/tcell_model/work/top_genes_list.csv", sep = ",", index_col = 0)
# Filter df to keep top DEGs
top_genes = top_genes_list["x"].tolist()
df = df.loc[top_genes] 

## Set independent and dependent variables and get gene IDs from columns

In [3]:
X = df.T
y = colData["state"] == "Th1"
gene_ids = X.columns

### 1. Feature selection (LASSO and RandomForest)

In [ ]:
# Set start time
start_time = time.perf_counter()

# Initialize leave-one-out cross validator
LOOCV = LeaveOneOut()

# Initialize lists for features per fold
features_lasso = []
features_rf = []

# Split using LOOCV
for fold_idx, (train_idx, test_idx) in enumerate(LOOCV.split(X)):

    # Filter X and y given training and testing indices
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    print("Starting fold:", fold_idx)

    # LASSO feature selection (per fold)
    print("Starting LASSO feature selection...")
    
    # Set estimator
    estimator_lasso = LogisticRegression(solver = "liblinear", penalty = "l1", max_iter = 300, random_state = 42)

    # Perform feature selection with LASSO estimator
    feature_selection_lasso = feature_sel(X_train, y_train, estimator_lasso)

    # Save features (per fold)
    features_lasso.append(feature_selection_lasso)
    
    # RandomForest feature selection (per fold)
    print("Starting RandomForest feature selection...")

    # Set estimator
    estimator_rf = RandomForestClassifier(n_estimators = 100, random_state = 42)

    # Perform feature selection with rf estimator
    feature_selection_rf = feature_sel(X_train, y_train, estimator_rf)

    # Save features (per fold)
    features_rf.append(feature_selection_rf) 

    print("Feature selection complete for fold:", fold_idx)
    print()

# Find the frequency of selected features and select the top 10

# Create counters
counter_lasso = collections.Counter()
counter_rf = collections.Counter()

# LASSO
for features in features_lasso:
    counter_lasso.update(features)

features_lasso_10 = [feature for feature, count in counter_lasso.most_common(10)]
features_lasso_10

# RandomForest
for features in features_rf:
    counter_rf.update(features)

features_rf_10 = [feature for feature, count in counter_rf.most_common(10)]
features_rf_10

# Print elapsed time
end_time = time.perf_counter()
print("Elapsed time:", end_time - start_time)

Starting fold: 0
Starting LASSO feature selection...
Starting RandomForest feature selection...


### 2. Model Training, Tuning, and Evaluation

In [68]:
# Set start time
start_time = time.perf_counter()

# Initialize leave-one-out cross validator
LOOCV = LeaveOneOut()

# Filter training and testing data to include selected features
X_lasso = X[features_lasso_10]
X_rf = X[features_rf_10]

# Initialize lists for storing accuracy scores per fold
acc_lr = []
acc_rf = []
acc_svm = []

acc_lasso_lr = []
acc_lasso_rf = []
acc_lasso_svm = []

acc_rf_lr = []
acc_rf_rf = []
acc_rf_svm = []


# Split using LOOCV
for fold_idx, (train_idx, test_idx) in enumerate(LOOCV.split(X)):

    # Filter X and y given training and testing indices 
    # for control (no feature selction), LASSO, and RandomForest
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    X_train_lasso, X_test_lasso = X_lasso.iloc[train_idx], X_lasso.iloc[test_idx]
    X_train_rf, X_test_rf = X_rf.iloc[train_idx], X_rf.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    print("Starting fold:", fold_idx)

    # Train models with no feature selection
    print("Starting model training with no feature selection...")
    models_crtl = model_train(X_train, y_train)
    
    # Train models with LASSO filtered data
    print("Starting model training with LASSO filtered data...")
    models_lasso = model_train(X_train_lasso, y_train)

    # Train models with RandomForest filtered data
    print("Starting model training with RandomForest filtered data...")
    models_rf = model_train(X_train_rf, y_train)
    
    # Evaluate control models
    acc_scores = model_eval(models_crtl, X_test, y_test)
    acc_lr.append(acc_scores[0])
    acc_rf.append(acc_scores[1])
    acc_svm.append(acc_scores[2])

    # Evaluate LASSO filtered models
    acc_scores_lasso = model_eval(models_lasso, X_test_lasso, y_test)
    acc_lasso_lr.append(acc_scores_lasso[0])
    acc_lasso_rf.append(acc_scores_lasso[1])
    acc_lasso_svm.append(acc_scores_lasso[2])

    # Evaluate RandomForest filtered models
    acc_scores_rf = model_eval(models_rf, X_test_rf, y_test)
    acc_rf_lr.append(acc_scores_rf[0])
    acc_rf_rf.append(acc_scores_rf[1])
    acc_rf_svm.append(acc_scores_rf[2])

    print()

# Print accuracy scores
print("Accuracy score for LogisticRegression using no feature selection:", round(np.mean(acc_lr), 3))
print("Accuracy score for RandomForest using no feature selection:", round(np.mean(acc_rf), 3))
print("Accuracy score for SVM using no feature selection:", round(np.mean(acc_svm), 3))
print()
print("Accuracy score for LogisticRegression using LASSO feature selection:", round(np.mean(acc_lasso_lr), 3))
print("Accuracy score for RandomForest using LASSO feature selection:", round(np.mean(acc_lasso_rf), 3))
print("Accuracy score for SVM using LASSO feature selection:", round(np.mean(acc_lasso_svm), 3))
print()
print("Accuracy score for LogisticRegression using RandomForest feature selection:", round(np.mean(acc_rf_lr), 3))
print("Accuracy score for RandomForest using RandomForest feature selection:", round(np.mean(acc_rf_rf), 3))
print("Accuracy score for SVM using RandomForest feature selection:", round(np.mean(acc_rf_svm), 3))
print()    

# Print elapsed time
end_time = time.perf_counter()
print("Elapsed time:", end_time - start_time)

Starting fold: 0
Starting model training with no feature selection
Starting model training with LASSO filtered data
Starting model training with RandomForest filtered data

Starting fold: 1
Starting model training with no feature selection
Starting model training with LASSO filtered data
Starting model training with RandomForest filtered data

Starting fold: 2
Starting model training with no feature selection
Starting model training with LASSO filtered data
Starting model training with RandomForest filtered data

Starting fold: 3
Starting model training with no feature selection
Starting model training with LASSO filtered data
Starting model training with RandomForest filtered data

Starting fold: 4
Starting model training with no feature selection
Starting model training with LASSO filtered data
Starting model training with RandomForest filtered data

Starting fold: 5
Starting model training with no feature selection
Starting model training with LASSO filtered data
Starting model trai